In [4]:
import numpy as np
import pandas as pd

In [468]:
# SNPchimp SNP information includes all of array type
df=pd.read_csv("arrays/SNPchimp_result_2583590585.csv")

/var/folders/sq/vc6_dvd116b6pt9nrzq_m1100000gr/T/ipykernel_39204/1573641335.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("arrays/SNPchimp_result_2583590585.csv")


In [469]:
df

,chip_name,rs,Alleles_A_B_FORWARD,Alleles_A_B_TOP,Alleles_A_B_Affymetrix,chromosome,position,SNP_name
0,Pig_Illu60Kv1,rs10719839,T/C,A/G,0/0,1,105951016,ALGA0005071
1,Pig_Illu60Kv1,rs45430995,A/G,A/G,0/0,1,286777913,ASGA0102985
2,Pig_Illu60Kv1,rs45435103,A/G,A/G,0/0,1,134715184,CAMB0000069
3,Pig_Illu60Kv1,rs45435473,T/C,A/G,0/0,1,286785390,ASGA0007005
4,Pig_Illu60Kv1,rs55618047,A/G,A/G,0/0,1,28424681,DIAS0004367
...,...,...,...,...,...,...,...,...
861045,Pig_Affy650k,NaN,0/0,0/0,A/G,99,0,AX-116661492
861046,Pig_Affy650k,NaN,0/0,0/0,A/C,99,0,AX-116661470
861047,Pig_Affy650k,NaN,0/0,0/0,A/G,99,0,AX-116661450
861048,Pig_Affy650k,NaN,0/0,0/0,A/G,99,0,AX-116661476


# Check which version the chip is
1. The meta information of some publish data is not complete, for example show the chip name but no chip version. 

In [470]:
# based on the SNP-name

In [471]:
uni_chip=df["chip_name"].unique()

In [472]:
corresp={}# key is file name, value is chip name
for j in ['dryad05qfttf0w', 'dryad30tk6', 
       'dryad905qfttnc', 'dryadv6f1g_DG_Boar', 'dryadv6f1g_DG_Pig',
       'figshare18318584','dryad53j31', 'dryad8bf48',]:
    bim=pd.read_csv(f'./arrays/{j}.bim',sep="\t",header=None)
    bim_SNP=bim[1]
    for i in uni_chip:
        chip_set=df.loc[df["chip_name"]==i,"SNP_name"]
        # Check if all values of bim_SNP are included in chip_set
        if set(bim_SNP).issubset(set(chip_set)):
            corresp.update({j:i})
            print(f"All SNPs in {j} are included in {i}")

All SNPs in dryad05qfttf0w are included in Pig_Illu60Kv2
All SNPs in dryad905qfttnc are included in Pig_Illu60Kv1
All SNPs in dryad905qfttnc are included in Pig_Illu60Kv2
All SNPs in dryadv6f1g_DG_Boar are included in Pig_Illu60Kv1
All SNPs in dryadv6f1g_DG_Pig are included in Pig_Illu60Kv1
All SNPs in figshare18318584 are included in Pig_Illu60Kv2
All SNPs in dryad53j31 are included in Pig_Illu60Kv1
All SNPs in dryad8bf48 are included in Pig_Illu60Kv1
All SNPs in dryad8bf48 are included in Pig_Illu60Kv2


In [473]:
# The array version of dryad30tk6 is not clear, temporarily as Pig_Illu60Kv1
corresp["dryad30tk6"]="Pig_Illu60Kv1"

# Correct the position

In [474]:
# v2 [dryad05qfttf0w,dryad30tk6,dryad905qfttnc,figshare18318584,
# v1 [dryad53j31,dryad8bf48,]
# intersection: dryadv6f1g_DG_Boar,dryadv6f1g_DG_Pig,figshare21971204CFJY_DD_1518,figshare21971204CFJY_LL_1702,figshare21971204CFJY_YY_4789,Z3816619
# rm chips that not meet requirement: [dryadsd784,dryad4gh70EHL_60K_gens,dryad4gh70Laiwu_60K_gens,dryad4gh70DLY_60K_gens]

In [475]:
for i in ['dryad05qfttf0w', 'dryad30tk6', 
       'dryad905qfttnc', 'dryadv6f1g_DG_Boar', 'dryadv6f1g_DG_Pig',
       'figshare18318584','dryad53j31', 'dryad8bf48',]:
    
    df1= df.loc[df["chip_name"]==corresp[i],:]
    df1.loc[df1["chromosome"]=="99","chromosome"]="MT"
    bim=pd.read_csv(f'./arrays/{i}.bim',sep="\t",header=None)
    bim_new=pd.merge(bim,df1,left_on=1,right_on="SNP_name",how="left")[["chromosome",1,2,"position",4,5]]
    bim_new.to_csv(f'./arrays/{i}_new.bim',header=False,index=False,sep="\t")

# Check the allele format

In [476]:
'dryad05qfttf0w',corresp['dryad05qfttf0w']

('dryad05qfttf0w', 'Pig_Illu60Kv2')

In [477]:
def fun_check_allele(x,column_name):
    if f"{x[4]}/{x[5]}"==x[column_name]:
        return 1  
    elif f"{x[5]}/{x[4]}"==x[column_name]:
        return 1
    else:
        return 0


In [478]:
def check_format(file_prefix,bim,df):
    df1=df.loc[df["chip_name"]==corresp[file_prefix],:]
    bim_df1=pd.merge(bim,df1,left_on=1,right_on="SNP_name",how="left")
    bim_df1["compare_Alleles_A_B_FORWARD"]=bim_df1.apply(lambda x: fun_check_allele(x, "Alleles_A_B_FORWARD"),axis=1)
    bim_df1["compare_Alleles_A_B_TOP"]=bim_df1.apply(lambda x: fun_check_allele(x, "Alleles_A_B_TOP"),axis=1)
    bim_df1["compare_Alleles_A_B_Affymetrix"]=bim_df1.apply(lambda x: fun_check_allele(x, "Alleles_A_B_Affymetrix"),axis=1)
    dic_number={"Alleles_A_B_FORWARD":bim_df1["compare_Alleles_A_B_FORWARD"].sum(),
        "Alleles_A_B_TOP":bim_df1["compare_Alleles_A_B_TOP"].sum(),
        "Alleles_A_B_Affymetrix":bim_df1["compare_Alleles_A_B_Affymetrix"].sum()}
    for i,j in dic_number.items():
        print(file_prefix,f": the number of SNPs fit {i} formate {j}")
    return max(dic_number.items(), key=lambda x: x[1])

In [479]:
dic_file_format={}# key: file name, value:allele format and number of insection
for i in ['dryad05qfttf0w', 'dryad30tk6', 
       'dryad905qfttnc', 'dryadv6f1g_DG_Boar', 'dryadv6f1g_DG_Pig',
       'figshare18318584','dryad53j31', 'dryad8bf48',]:
    bim=pd.read_csv(f'./arrays/{i}.bim',sep="\t",header=None)
    a=check_format(i,bim,df)
    dic_file_format[i]=a

dryad05qfttf0w : the number of SNPs fit Alleles_A_B_FORWARD formate 49384
dryad05qfttf0w : the number of SNPs fit Alleles_A_B_TOP formate 25672
dryad05qfttf0w : the number of SNPs fit Alleles_A_B_Affymetrix formate 1844
dryad30tk6 : the number of SNPs fit Alleles_A_B_FORWARD formate 58514
dryad30tk6 : the number of SNPs fit Alleles_A_B_TOP formate 31772
dryad30tk6 : the number of SNPs fit Alleles_A_B_Affymetrix formate 0
dryad905qfttnc : the number of SNPs fit Alleles_A_B_FORWARD formate 24903
dryad905qfttnc : the number of SNPs fit Alleles_A_B_TOP formate 47868
dryad905qfttnc : the number of SNPs fit Alleles_A_B_Affymetrix formate 0
dryadv6f1g_DG_Boar : the number of SNPs fit Alleles_A_B_FORWARD formate 30674
dryadv6f1g_DG_Boar : the number of SNPs fit Alleles_A_B_TOP formate 15904
dryadv6f1g_DG_Boar : the number of SNPs fit Alleles_A_B_Affymetrix formate 0
dryadv6f1g_DG_Pig : the number of SNPs fit Alleles_A_B_FORWARD formate 43218
dryadv6f1g_DG_Pig : the number of SNPs fit Alleles_A

In [480]:
dic_file_format

{'dryad05qfttf0w': ('Alleles_A_B_FORWARD', 49384),
 'dryad30tk6': ('Alleles_A_B_FORWARD', 58514),
 'dryad905qfttnc': ('Alleles_A_B_TOP', 47868),
 'dryadv6f1g_DG_Boar': ('Alleles_A_B_FORWARD', 30674),
 'dryadv6f1g_DG_Pig': ('Alleles_A_B_FORWARD', 43218),
 'figshare18318584': ('Alleles_A_B_TOP', 50936),
 'dryad53j31': ('Alleles_A_B_FORWARD', 0),
 'dryad8bf48': ('Alleles_A_B_FORWARD', 47287)}

# dryad53j31 allele format is special
the coding is 1,2,3,4, should correspond to ACGT

In [481]:
bim=pd.read_csv(f'./arrays/dryad53j31.bim',sep="\t",header=None)

In [482]:
bim[4]=bim[4].replace(1,"A")
bim[4]=bim[4].replace(2,"C")
bim[4]=bim[4].replace(3,"G")
bim[4]=bim[4].replace(4,"T")

In [483]:
bim[5]=bim[5].replace(1,"A")
bim[5]=bim[5].replace(2,"C")
bim[5]=bim[5].replace(3,"G")
bim[5]=bim[5].replace(4,"T")

In [484]:
bim.to_csv(f'./arrays/dryad53j31_new.bim',header=False,index=False,sep="\t")

In [485]:
df1=df.loc[df["chip_name"]==corresp['dryad53j31'],:]
bim_df1=pd.merge(bim,df1,left_on=1,right_on="SNP_name",how="left")
bim_df1["compare_Alleles_A_B_FORWARD"]=bim_df1.apply(lambda x: fun_check_allele(x, "Alleles_A_B_FORWARD"),axis=1)
bim_df1["compare_Alleles_A_B_TOP"]=bim_df1.apply(lambda x: fun_check_allele(x, "Alleles_A_B_TOP"),axis=1)
bim_df1["compare_Alleles_A_B_Affymetrix"]=bim_df1.apply(lambda x: fun_check_allele(x, "Alleles_A_B_Affymetrix"),axis=1)
print("dryad53j31",": the number of SNPs fit Alleles_A_B_FORWARD formate",bim_df1["compare_Alleles_A_B_FORWARD"].sum())
print("dryad53j31",": the number of SNPs fit Alleles_A_B_TOP formate",bim_df1["compare_Alleles_A_B_TOP"].sum())
print("dryad53j31",": the number of SNPs fit Alleles_A_B_Affymetrix formate",bim_df1["compare_Alleles_A_B_Affymetrix"].sum())


dryad53j31 : the number of SNPs fit Alleles_A_B_FORWARD formate 27506
dryad53j31 : the number of SNPs fit Alleles_A_B_TOP formate 52556
dryad53j31 : the number of SNPs fit Alleles_A_B_Affymetrix formate 0


In [486]:
dic_file_format["dryad53j31"]=('Alleles_A_B_TOP', 52556)

# Check whether different format flip the allele A and B
This step is for checking whether the two versin's SNP ref and alt allele is consistent.

In [487]:
def fun_check(x,c1,c2):
    dic={"A":"T","T":"A","C":"G","G":"C"}
    a,b=x[c1].split("/")
    if f"{a}/{b}"==x[c2]:
        return 1  
    elif f"{b}/{a}"==x[c2]:
        return 1
    elif f"{dic[a]}/{dic[b]}"==x[c2]:
        return 2
    elif f"{dic[b]}/{dic[a]}"==x[c2]:
        return 2
    else:
        return 0
df1.apply(lambda x: fun_check(x, "Alleles_A_B_FORWARD","Alleles_A_B_TOP"),axis=1)

0         2
1         1
2         1
3         2
4         1
         ..
860776    2
860777    2
860778    1
860779    2
860780    1
Length: 62163, dtype: int64

In [488]:
from collections import Counter
Counter(df1.apply(lambda x: fun_check(x, "Alleles_A_B_FORWARD","Alleles_A_B_TOP"),axis=1))

Counter({1: 32450, 2: 29713})

In [489]:
Counter(df1.apply(lambda x: fun_check(x, "Alleles_A_B_FORWARD","Alleles_A_B_Affymetrix"),axis=1))

Counter({0: 62163})

In [490]:
Counter(df1.apply(lambda x: fun_check(x, "Alleles_A_B_TOP","Alleles_A_B_Affymetrix"),axis=1))

Counter({0: 62163})

# Convert to consistent version

In [491]:
# def fun_flip_allele(x,Input,Output):
#     dic={"A":"T","T":"A","C":"G","G":"C","0":"0"}
#     if f"{dic[x[4]]}/{dic[x[5]]}"==x[Output]:
#         x[4]=dic[x[4]]
#         x[5]=dic[x[5]]
#     elif f"{dic[x[5]]}/{dic[x[4]]}"==x[Output]:
#         x[4]=dic[x[4]]
#         x[5]=dic[x[5]]
#     else:
#         pass
#     return x

In [492]:
def fun_flip_allele(x, Input, Output, missing = ["0", "-"]):
    Input = x[Input].split("/")
    Output = x[Output].split("/")

    result = []

    for allele in [x[4], x[5]]:

        if allele in missing:
            result.append("0")

        elif allele not in Input:
            raise Exception(f"Allele {allele} not in {Input}")

        else:
            result.append(Output[Input.index(allele)])

    x[4], x[5] = result

    return x

In [493]:
def convert(file,origin_format,output_format,SNPchiMp_table,bim):
    df1=SNPchiMp_table.loc[SNPchiMp_table["chip_name"]==corresp[file],:]
    bim_df1=pd.merge(bim,df1,left_on=1,right_on="SNP_name",how="left")
    bim_df1=bim_df1.apply(lambda x: fun_flip_allele(x,origin_format,output_format),axis=1)
    bim_new=bim_df1[[0,1,2,3,4,5]]
    a=check_format(file,bim_new,SNPchiMp_table)
    print(a)
    return bim_new

In [494]:
dic_file_format

{'dryad05qfttf0w': ('Alleles_A_B_FORWARD', 49384),
 'dryad30tk6': ('Alleles_A_B_FORWARD', 58514),
 'dryad905qfttnc': ('Alleles_A_B_TOP', 47868),
 'dryadv6f1g_DG_Boar': ('Alleles_A_B_FORWARD', 30674),
 'dryadv6f1g_DG_Pig': ('Alleles_A_B_FORWARD', 43218),
 'figshare18318584': ('Alleles_A_B_TOP', 50936),
 'dryad53j31': ('Alleles_A_B_TOP', 52556),
 'dryad8bf48': ('Alleles_A_B_FORWARD', 47287)}

In [495]:
Input_format='Alleles_A_B_TOP'
Output_format='Alleles_A_B_FORWARD'
for i in dic_file_format:
    if dic_file_format[i][0]==Output_format:
        pass
    else:
        print("Finish converting")
        bim=pd.read_csv(f'./arrays/{i}_new.bim',sep="\t",header=None)
        bim_new=convert(i,dic_file_format[i][0],Output_format,df,bim)
        bim_new.to_csv(f'./arrays/{i}_new.bim',header=False,index=False,sep="\t")

Finish converting
dryad905qfttnc : the number of SNPs fit Alleles_A_B_FORWARD formate 47868
dryad905qfttnc : the number of SNPs fit Alleles_A_B_TOP formate 24903
dryad905qfttnc : the number of SNPs fit Alleles_A_B_Affymetrix formate 0
('Alleles_A_B_FORWARD', 47868)
Finish converting
figshare18318584 : the number of SNPs fit Alleles_A_B_FORWARD formate 50936
figshare18318584 : the number of SNPs fit Alleles_A_B_TOP formate 26705
figshare18318584 : the number of SNPs fit Alleles_A_B_Affymetrix formate 2246
('Alleles_A_B_FORWARD', 50936)
Finish converting
dryad53j31 : the number of SNPs fit Alleles_A_B_FORWARD formate 52556
dryad53j31 : the number of SNPs fit Alleles_A_B_TOP formate 27506
dryad53j31 : the number of SNPs fit Alleles_A_B_Affymetrix formate 0
('Alleles_A_B_FORWARD', 52556)


# Compare the difference of version's SNP position

In [34]:
def correspond(x):
    if x["chromosome_x"]==x["chromosome_y"]:
        if x["position_x"]==x["position_y"]:
            return "T"
        else:
            return "F"
    else:
        return "F"
df["whether correspond"]=df.apply(correspond,axis=1)

In [48]:
temp=df[df["whether correspond"]=="F"]

In [51]:
temp[temp["chromosome_x"]!="99"]

,chip_name_x,rs_x,chromosome_x,position_x,SNP_name,chip_name_y,rs_y,chromosome_y,position_y,whether correspond
49466,Pig_Illu60Kv1,rs80894900,99,0.0,ALGA0010261,NaN,NaN,NaN,NaN,F
49485,Pig_Illu60Kv1,rs80920939,99,0.0,ASGA0002183,NaN,NaN,NaN,NaN,F
49495,Pig_Illu60Kv1,rs80937102,99,0.0,ASGA0004474,NaN,NaN,NaN,NaN,F
49530,Pig_Illu60Kv1,rs80976074,99,0.0,ALGA0008942,NaN,NaN,NaN,NaN,F
49592,Pig_Illu60Kv1,rs81348331,99,0.0,ALGA0001100,NaN,NaN,NaN,NaN,F
...,...,...,...,...,...,...,...,...,...,...
60521,Pig_Illu60Kv1,rs81341098,99,0.0,ALGA0112495,NaN,NaN,NaN,NaN,F
60609,Pig_Illu60Kv1,rs81342473,99,0.0,ALGA0113663,NaN,NaN,NaN,NaN,F
60620,Pig_Illu60Kv1,rs81266480,99,0.0,MARC0083084,NaN,NaN,NaN,NaN,F
60676,Pig_Illu60Kv1,NaN,99,0.0,DRGA0007336,NaN,NaN,NaN,NaN,F


In [ ]:
# 60k chip v1 and v2's intersection don't have difference on chromosome and position

# compare dryad905qfttnc_new.bim and dryad905qfttnc_converted.bim

In [496]:
bim=pd.read_csv(f'./arrays/dryad905qfttnc_new.bim',sep="\t",header=None)

In [497]:
bim2=pd.read_csv(f'./arrays/dryad905qfttnc_converted.bim',sep="\t",header=None)

In [505]:
bim[1]

0        MARC0044150
1        ASGA0000014
2        ASGA0000021
3        ALGA0000009
4        ALGA0000014
            ...     
47866    CASI0001308
47867    ASGA0080449
47868    M1GA0023446
47869    ALGA0120811
47870    ALGA0104570
Name: 1, Length: 47871, dtype: object

In [506]:
bim2[1]

0        ASGA0000014
1        ALGA0000009
2        ALGA0000014
3        H3GA0000032
4        INRA0000015
            ...     
47866    DRGA0017057
47867    ALGA0119800
47868    ASGA0090395
47869    MARC0056150
47870    MARC0112864
Name: 1, Length: 47871, dtype: object

In [516]:
df=pd.merge(bim,bim2,how="outer",on=1)

In [520]:
df

,0_x,1,2_x,3_x,4_x,5_x,0_y,2_y,3_y,4_y,5_y
0,MT,MARC0044150,0,0,T,C,26,0,0,T,C
1,1,ASGA0000014,0,342481,A,C,1,0,342481,A,C
2,99,ASGA0000021,0,0,G,T,26,0,0,G,T
3,1,ALGA0000009,0,538161,A,G,1,0,538161,A,G
4,1,ALGA0000014,0,565627,T,C,1,0,565627,T,C
...,...,...,...,...,...,...,...,...,...,...,...
47866,18,CASI0001308,0,60484331,T,C,18,0,60484331,T,C
47867,18,ASGA0080449,0,60518659,T,C,18,0,60518659,T,C
47868,18,M1GA0023446,0,60537055,A,C,18,0,60537055,A,C
47869,18,ALGA0120811,0,60805322,G,A,18,0,60805322,G,A


In [518]:
np.sum(df.apply(lambda x : 0 if x["4_x"] ==x["4_y"]  else 1, axis=1))

0

In [519]:
np.sum(df.apply(lambda x : 0 if x["5_x"] ==x["5_y"]  else 1, axis=1))

0